#### Names of people in the group

Please write the names of the people in your group in the next cell.

Elizabeth Pan

Magnus Sagmo

In [ ]:
# Loading modules that we need
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

# Add your imports below this line
import pyspark.sql.functions as F

In [ ]:
# A helper function to load a table (stored in Parquet format) from DBFS as a Spark DataFrame 
def load_df(table_name: "name of the table to load") -> DataFrame:
    return spark.read.parquet(table_name)

users_df = load_df("/user/hive/warehouse/users")
posts_df = load_df("/user/hive/warehouse/posts")

# Uncomment if you need
# comments_df = load_df("/user/hive/warehouse/comments")
# badges_df = load_df("/user/hive/warehouse/badges")

#### The problem: mining the interests of experts

The primary role of a questions and answering platform such as Stack Exchange is to connect two types of people. Namely, people who have questions in areas such as computer science or data science and knowledgeable people who can answer those questions reliably. Let's call the first category of people' knowledge seekers' and the second one 'expert users' or 'experts' for short.

Here we want to answer a question related to the diversity of topics that experts are interested in using our data. We want to know if expert users only answer questions in a specific set of topics or their interests include a wide variety of topics.

To answer the above question, we will compute the correlation between a user's expertise level and the diversity of topics of questions they have answered. The first step is to define two variables (or measures); first for 'user expertise level' and then for 'user interest diversity'. Then we will use the Pearson correlation coefficient to measure the linear correlation between the two variables. We define the variables as:

   - VariableA (the measure of user expertise level). We will use the 'Reputation' column from 'users' table, which according to Stack Exchange's documentation "is a rough measurement of how much the community trusts you; it is earned by convincing your peers that you know what you're talking about" as an indicator of a user's expertise level on the platform. 

   - VariableB (The measure of user interest diversity). We measure the diversity of a user's interests by computing the total number of distinct tags associated with the questions each user has answered divided by the total number of unique tags which is 638.

Compute the Pearson correlation coefficient between VariableA and VariableB, and based on the result you've got, answer the following question: 

     Do expert users have specif interests or do they have general interests?

Please explain your reasoning on how you reached your answer.

You should use Apache Spark API for your implementation. You can use the Spark implementation of the Pearson correlation coefficient.

In [ ]:
def var_A(users_df):
  return users_df.select("Id", "Reputation")

In [ ]:
UNIQUE_TAGS = 638

def diversity_number(concatenated_tags):
  return concatenated_tags.count("<")

new_f = F.udf(diversity_number, IntegerType())
  
def var_B(posts_df):
  # get all answers
  answer_posts = posts_df.alias("a").select("Id", "ParentId", "OwnerUserId").filter(col("PostTypeId") == 2)
  question_posts = posts_df.alias("q").select("Id", "Tags").filter(col("PostTypeId") == 1)
  joined = answer_posts.join(question_posts, col("a.ParentId") == col("q.Id")).select("q.Id", "a.OwnerUserId", "q.tags")
  concatenated = joined.groupby("a.OwnerUserId").agg(concat_ws("", collect_list(col("q.Tags"))).alias("ConcatenatedTags"))
  concatenated_w_number = concatenated.withColumn("NoOfTags", new_f("ConcatenatedTags")).select("a.OwnerUserId", "NoOfTags")
  
  return concatenated_w_number

In [ ]:
#Compute the Pearson correlation coefficient between VariableA and VariableB, and based on the result you've got, answer the following question: 
df_reputation = var_A
df_posts = var_B

def pearson(df_A, df_B):
  df_joined = df_B.join(df_A, df_B.OwnerUserId == df_A.Id)
  return df_joined.corr("Reputation", "NoOfTags")
  
pearson_df = pearson(var_A(users_df), var_B(posts_df))
print(pearson_df)

0.7724926022809941


_Do expert users have specif interests or do they have general interests?_

The Pearson Correlation Coefficient between the user expertise level and the user interest diversity, $PCC=0.772$. PCC ranges from -1 to 1, and the closer a value is to 1, the more correlated are the two variables. Considering our value of 0.772, we conclude that expert users (users with high reputation) are likely to have general interests (high interest diversity).
